a.	Сгенерите датафрейм размера 4х10 и наполните его случайными целыми числами в промежутке [1,10], выведите его на экран 

b.	Проведите преобразование: Для тех строк, в которых число во 2 колонке больше 3, поделите число в 4 колонке на число из 2 колонки

c.	Выведите получившийся датафрейм на экран, посчитайте разницу сумм колонок между исходным датафреймом и тем, что получился после преобразований

In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.DataFrame(np.random.randint(1, 10, size=(10, 4)))

In [3]:
df1 = df.copy()
df1

,0,1,2,3
0,7,6,8,1
1,8,3,7,8
2,8,9,1,1
3,1,1,4,2
4,8,6,6,4
5,1,4,5,6
6,2,8,7,9
7,5,5,1,4
8,7,5,6,6
9,5,1,4,7


In [4]:
mask = df1[1] > 3

In [5]:
df1.loc[mask, 3] = (df1.loc[mask, 3] / df1.loc[mask, 1]).astype(int)

In [6]:
print(df, df1, sep='\n\n')

   0  1  2  3
0  7  6  8  1
1  8  3  7  8
2  8  9  1  1
3  1  1  4  2
4  8  6  6  4
5  1  4  5  6
6  2  8  7  9
7  5  5  1  4
8  7  5  6  6
9  5  1  4  7

   0  1  2  3
0  7  6  8  0
1  8  3  7  8
2  8  9  1  0
3  1  1  4  2
4  8  6  6  0
5  1  4  5  1
6  2  8  7  1
7  5  5  1  0
8  7  5  6  1
9  5  1  4  7


In [7]:
df.sum() - df1.sum()

0     0
1     0
2     0
3    28
dtype: int64

Откройте архивированный csv файл с данными. Там находятся логи о посещениях страниц сайта. Страницы находятся в поле ‘event’, событие event_9 считается целевым (покупка). Событие ‘lost’ означает уход пользователя с сайта. Вам нужно:

i.	Вычислите средний промежуток времени между сессиями (в минутах)

In [8]:
data = pd.read_csv(r'./test_task.csv.gz', parse_dates=['timestamp'])

In [9]:
data['next_time'] = data['timestamp'].shift(-1)

In [10]:
data.head(10)

,client_id,session,event,timestamp,next_time
0,136651,1366511,sleep1->event_1,2020-05-01 00:00:00.000000000,2020-05-01 00:00:00.199838279
1,136651,1366511,sleep1->event_2,2020-05-01 00:00:00.199838279,2020-05-01 00:00:00.729752253
2,136651,1366511,sleep1->event_3,2020-05-01 00:00:00.729752253,2020-05-01 00:00:01.263877058
3,136651,1366511,sleep1->event_6,2020-05-01 00:00:01.263877058,2020-05-01 00:00:01.804570372
4,136651,1366511,lost,2020-05-01 00:00:01.804570372,2020-05-01 00:00:00.000000000
5,136652,1366521,sleep1->event_1,2020-05-01 00:00:00.000000000,2020-05-01 00:00:00.421842598
6,136652,1366521,lost,2020-05-01 00:00:00.421842598,2020-05-01 00:00:00.000000000
7,136653,1366531,sleep1->event_1,2020-05-01 00:00:00.000000000,2020-05-01 00:00:00.595333456
8,136653,1366531,sleep1->event_2,2020-05-01 00:00:00.595333456,2020-05-01 00:00:00.858637468
9,136653,1366531,sleep1->event_3,2020-05-01 00:00:00.858637468,2020-05-01 00:00:01.457439707


In [11]:
temp = data.groupby('session')[['session', 'timestamp', 'next_time']].last()

In [12]:
temp.head()

,session,timestamp,next_time
session,,,
1366511,1366511,2020-05-01 00:00:01.804570372,2020-05-01 00:00:00.000000
1366521,1366521,2020-05-01 00:00:00.421842598,2020-05-01 00:00:00.000000
1366531,1366531,2020-05-01 00:00:02.988981776,2020-05-10 04:09:19.106087
1366532,1366532,2020-05-10 04:09:21.925408898,2020-05-01 00:00:00.000000
1366541,1366541,2020-05-01 00:00:03.048044657,2020-05-01 00:00:00.000000


In [13]:
(temp.next_time - temp.timestamp).mean()

Timedelta('0 days 00:02:05.895992')

ii.	Вычислите средний промежуток времени между событиями внутри сессии (в микросекундах)

In [14]:
data.groupby('session')['timestamp'].diff().mean()

Timedelta('0 days 00:00:00.431372')

iii.	Мы хотели бы узнать какие события чаще встречаются в «плохих» сессиях (целевого события не случилось), а какие чаще в «хороших» (целевое событие случилось). Для каждого события посчитайте:
1.	Кол-во хороших сессий, в которых присутствовало данное событие
2.	Кол-во плохих сессий, в которых присутствовало данное событие
3.	Какую информацию можно извлечь из этих двух полей? Можно ли на их основе рассчитать показатель, который бы помогал выделять события, сильно связанные с хорошим/плохим исходом? 


In [15]:
q = data.loc[data.event.str.contains('event_9')].groupby('session')['event'].count()

In [16]:
data['status'] = pd.Series(np.where(data.session.isin(q.index), 'positive', 'negative'))

In [17]:
data.groupby('status')['session'].count()

status
negative    44670
positive    18851
Name: session, dtype: int64